# Crear _wrapper_

Creamos un _wrapper_ que modifica las salidas de un modelo **ya entrenado** para que sea compatible con el formato `numpy`:

In [31]:
from sklearn.base import BaseEstimator, RegressorMixin

class KerasRegressorWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, model):
        self.model = model

    def fit(self, X, y):
        # Asume que el modelo ya está entrenado, por lo que no hacemos nada en fit
        return self

    def predict(self, X):
        return self.model.predict(X).flatten()

# Cargar modelo

Cargamos el modelo ya entrenado y le aplicamos el _wrapper_:

In [32]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model_tf = load_model("model.keras")
model = KerasRegressorWrapper(model_tf)

# Cargar datos

Cargamos los datos de **calibración** y *_test_*:

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

data = fetch_california_housing()
X, y = data.data, data.target

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_cal, X_test, y_cal, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Cálculo de incertidumbre

Utilizamos `MapieRegressor`para calcular los intervalos de las predicciones:

In [37]:
from mapie.regression import MapieRegressor

mapie_regressor = MapieRegressor(estimator=model)

mapie_regressor.fit(X_cal, y_cal)
y_preds, y_intervals = mapie_regressor.predict(X_test, alpha=[0.05, 0.1])

104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 473us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 412us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 420us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 410us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 416us/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 0s 406us/step


Vemos algunos ejemplos:

In [38]:
for i in range(5):
    print(f"Predicción: {y_preds[i]:.2f} | Intervalo: {y_intervals[i].flatten()}\t | Amplitud: {y_intervals[i][1] - y_intervals[i][0]}")


Predicción: 0.39 | Intervalo: [-0.63136146 -0.35770522  1.40228472  1.12862848]	 | Amplitud: [2.03364618 1.4863337 ]
Predicción: 0.92 | Intervalo: [-0.09375629  0.17989995  1.93988988  1.66623364]	 | Amplitud: [2.03364618 1.4863337 ]
Predicción: 4.87 | Intervalo: [3.85505077 4.12870701 5.88869695 5.6150407 ]	 | Amplitud: [2.03364618 1.4863337 ]
Predicción: 2.73 | Intervalo: [1.71565338 1.98930962 3.74929956 3.47564332]	 | Amplitud: [2.03364618 1.4863337 ]
Predicción: 2.68 | Intervalo: [1.66110798 1.93476422 3.69475416 3.42109792]	 | Amplitud: [2.03364618 1.4863337 ]


## Código de prueba

In [36]:
# import mapie
# import pandas as pd

# from sklearn.datasets import fetch_california_housing
# from sklearn.model_selection import train_test_split

# X, y = fetch_california_housing(return_X_y=True, as_frame=True)
# X_train_and_cal, X_test, y_train_and_cal, y_test = train_test_split(X, y, test_size=1/3)

# y_test_pred_interval = pd.DataFrame(mapie_regressor.predict(X_test, alpha=.05)[1].reshape(-1,2), index=X_test.index, columns=["left", "right"])
# y_test_pred_interval